# Basic System Evaluation

To fully understand the task, and the difficulties, I'll run one pass of evaluation in this NB. After the initial exploration, we can productionize this code.

In [7]:
# Imports
import sklearn
from sklearn.metrics import average_precision_score

In [28]:
# loading datasets

sub_tasks = ['celebrity', 'movie', 'restaurant', 'tvShow']
trainset_locs = ['../data/TRAIN SET/%s.TRAINSET.txt' % t for t in sub_tasks]
devset_locs = ['../data/DEV SET/%s.DEVSET.txt' % t for t in sub_tasks]

# take celebrity as example
lines = unicode(open(trainset_locs[0]).read(), 'gbk').split('\n')
parsing_result = []
for line in lines:
    terms = line.split('\t')
    items = []
    for i in terms[1:]:
        ent, score = tuple(i.split(':'))
        items.append((ent, int(score)))
    if len(items) == 0:
        continue
    parsing_result.append((terms[0], items))

In [21]:
print parsing_result[0]

(u'\u6c11\u56fd\u9769\u547d\u5bb6', [(u'\u9ec4\u4e43\u88f3', 1), (u'\u6851\u539f\u548c\u771f', 0), (u'\u738b\u5fec', 0), (u'\u5f20\u79b9(\u73b0\u4ee3\u77e5\u540d\u4f5c\u5bb6\u3001\u6587\u5b66\u8bc4\u8bba\u5bb6\u3001\u7f16\u8f91\u5bb6)', 0), (u'\u65b9\u60f3', 0), (u'\u5218\u6770(\u4e0a\u6d77\u793e\u4f1a\u79d1\u5b66\u9662\u535a\u5bfc)', 0), (u'\u5f20\u8f89(\u7518\u8083\u7701\u56fd\u571f\u8d44\u6e90\u5385\u526f\u5de1\u89c6\u5458)', 0), (u'\u6881\u51a0\u534e', 0), (u'\u9646\u5b50\u4fee(\u8457\u540d\u4e09\u519c\u95ee\u9898\u4e13\u5bb6)', 0), (u'\u674e\u9640', 0), (u'\u6768\u518d\u5e73', 0), (u'\u5f90\u83b9(\u4e2d\u56fd\u4eba\u6c11\u5927\u5b66\u526f\u6559\u6388)', 0), (u'\u8d1e\u513f\u7687\u540e', 0), (u'\u6c88\u5955\u6590', 0), (u'\u5f20\u5b66\u5175(\u5bbf\u8fc1\u5e02\u6cd5\u5b98\u57f9\u8bad\u5b66\u9662\u515a\u59d4\u59d4\u5458\u3001\u526f\u9662\u957f)', 0), (u'\u6768\u541b(\u5929\u6d25\u677e\u6c5f\u8db3\u7403\u4ff1\u4e50\u90e8\u8fd0\u52a8\u5458)', 0), (u'\u9976\u4e91\u6c5f', 0), (u'\u7434\u

In [32]:
def NoReorder(q, results):
    return results

def Reversed(q, results):
    return reversed(results)

def EvaluateByRank(strategy):
    score_results = []
    for query, gs_result in parsing_result:
        my_result = strategy(query, [i for (i, t) in gs_result])
        rank_map = dict((r, i) for (i, r) in enumerate(my_result))
        l = len(gs_result)
        scores = [float(l-rank_map[i]) / l for (i, t) in gs_result]
        s = average_precision_score([t for i, t in gs_result], scores)
        score_results.append(s)
    return sum(score_results) / len(score_results)

print EvaluateByRank(NoReorder)
print EvaluateByRank(Reversed)

0.999258594998
0.0649129504582


In [37]:
def OrderByScore(func):
    def wrappee(q, results):
        return [r for s, r in sorted([
                    (func(q, r), r) for r in results
                ], reverse=True)]
    return wrappee

# Try a real metric
@OrderByScore
def CharOverlap(q, r):
    return len(set(q).intersection(set(r)))

print EvaluateByRank(CharOverlap)
    

0.297675985437


In [ ]:
# exporting results into a file
